In [1]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.agents import AgentType
from langchain.agents import Tool, initialize_agent
import chromadb
from langchain.embeddings import OpenAIEmbeddings
import pandas as pd
import requests

In [ ]:
api_key = ""
llm = ChatOpenAI(temperature=0.7, openai_api_key=api_key, model='gpt-4')
embeddings = OpenAIEmbeddings(openai_api_key=api_key)
chain =  llm

C:\Users\mati1\AppData\Local\Temp\ipykernel_20160\4081575595.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=api_key)


In [4]:
client = chromadb.PersistentClient(path=".")
doc_collection = client.get_or_create_collection(name="hidrogeno_verde")

In [5]:
def store_in_chromadb(documents, metadata_list):
    for idx, (doc, metadata) in enumerate(zip(documents, metadata_list)):
        embedding = embeddings.embed_query(doc)  # Genera el embedding
        doc_collection.add(
            ids=[str(idx)],  # ID único para cada documento
            documents=[doc],  # Documento en formato texto
            embeddings=[embedding],  # Embedding generado
            metadatas=[metadata]  # Información adicional (opcional)
        )


In [ ]:
#Se puede omitir si ya tiene cargados los datos
df = pd.read_csv('hidrogeno_verde.csv')
df['date'] = pd.to_datetime(df['date'], format='%b %d, %Y @ %H:%M:%S.%f')
df['date'] = df['date'].dt.strftime('%Y-%m-%d')
df['combined'] = df['title'] + ' ' + df['text']
documents = df['text'].tolist() 
metadata_list = df.drop(columns=['text']).to_dict(orient='records') 

# Almacenar en ChromaDB
store_in_chromadb(documents, metadata_list)

In [6]:
# Función para realizar la búsqueda en ChromaDB
def search_in_chromadb(query):
    # Realiza la búsqueda en ChromaDB utilizando la consulta
    results = doc_collection.query(
        query_embeddings=embeddings.embed_query(query),
        n_results=3  # Número de resultados a obtener
    )
    return results

In [7]:
def search_tool(query):
    search_results = search_in_chromadb(query)
    return "\n".join([str(doc) for doc in search_results['documents']])

# Función para búsqueda en Wikipedia
def wikipedia_search(query):
    response = requests.get(f"https://es.wikipedia.org/w/api.php", params={
        "action": "query",
        "format": "json",
        "titles": query,
        "prop": "extracts",
        "exintro": True
    })
    pages = response.json()['query']['pages']
    page = next(iter(pages.values()))
    return page.get('extract', "No se encontró información en Wikipedia.")
# Crear herramientas
tools = [
    Tool(
        name="ChromaDB Search",
        func=search_tool,
        description="Busca en la base de datos de ChromaDB los documentos relevantes."
    ),
    Tool(
        name="Wikipedia Search",
        func=wikipedia_search,
        description="Consulta Wikipedia para obtener más información sobre el tema."
    )
]
prompt_template = """
Eres un agente que puede interactuar con herramientas para obtener información relevante para responder preguntas. 
Tienes acceso a dos herramientas:

1. **ChromaDB Search**: Busca en una base de datos de documentos para obtener información relevante.
2. **Wikipedia Search**: Realiza una consulta a Wikipedia para obtener un resumen de un tema.

Cuando recibas una pregunta, debes decidir qué herramienta usar y cómo responder de manera concisa y precisa. 
Si es posible, utiliza primero la búsqueda en ChromaDB, si no encuentras suficiente información, consulta Wikipedia.
"""

# Crear el PromptTemplate
prompt = PromptTemplate(input_variables=["tools", "query", "agent_scratchpad", "tool_names"], template=prompt_template)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)




C:\Users\mati1\AppData\Local\Temp\ipykernel_20160\4060135420.py:43: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(


In [8]:
# Ejecutar el agente con una consulta usando `invoke`
query = "¿Qué déclaraciones ha realizado Juan Carlos Jobet en el contexto del hidrógeno verde?"
response = agent.invoke(query)

# Mostrar la respuesta
print(response)




> Entering new AgentExecutor chain...
Primero necesito buscar información relevante sobre las declaraciones de Juan Carlos Jobet en relación al hidrógeno verde.
Action: ChromaDB Search
Action Input: Declaraciones de Juan Carlos Jobet sobre hidrógeno verde
Observation: ['– Eliana Ximena Oyarzún Nancuante  “Les aseguro que este plan no va a quedar guardado en un cajón”, expresó el presidente Gabriel Boric, al momento de recibir el Plan de Acción de Hidrógeno Verde estructurado por un comité de diez expertos encabezados por el ex ministro de Energía del segundo gobierno de Sebastián Piñera, Juan Carlos Jobet. En su discurso la máxima autoridad nacional señaló que esta industria es una tremenda oportunidad que no se puede desaprovechar y que “el hidrógeno verde va a darnos mejores condiciones de vida”. El Mandatario confidenció que este es uno de los temas que mayor interés suscita en cada audiencia que tiene con líderes mundiales durante sus giras por Europa y Asia. En la ceremonia estu